# Pandas avançado

### Dados categorizados

Esta seção apresenta o tipo *Categorical* do pandas. Mostrarei como é
possível alcançar melhor desempenho e uso de memória em
algumas operações do pandas utilizando esse tipo. Apresentarei
também algumas ferramentas para utilizar dados categorizados em
aplicações estatísticas e de aprendizado de máquina (machine
learning).

### Informações básicas e motivação

Com frequência, uma coluna em uma tabela pode conter instâncias
repetidas de um conjunto menor de valores distintos. Já vimos
funções como *unique* e *value_counts*, que nos permitem extrair os
valores distintos de um array e calcular suas frequências,
respectivamente:

In [1]:
import pandas as pd
import numpy as np

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

Muitos sistemas de dados – para armazém de dados (data
warehousing), cálculos estatísticos ou outros usos – desenvolveram
abordagens especializadas para representar dados com valores
repetidos visando a uma armazenagem e a um processamento mais
eficazes. Em armazéns de dados, uma boa prática consiste em
utilizar as chamadas *tabelas de dimensão* (dimension tables), que
contêm os valores distintos, e armazenar os principais dados
observados como chaves inteiras que referenciam a tabela de
dimensão :

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)

In [7]:
dim = pd.Series(['apple', 'orange'])

In [8]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

Podemos usar o método *take* para restaurar a Series original de
strings:

In [10]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

Essa representação na forma de inteiros chama-se representação
*categorizada* (categorical) ou *codificada em dicionários* (dictionary-
encoded). O array de valores distintos pode ser chamado de
*categorias, dicionário ou níveis dos dados*. Neste livro, usaremos os
termos *categorizado e categorias*. Os valores inteiros que referenciam
as categorias são chamados de *códigos de categoria* ou simplesmente
*códigos*.
A representação em categorias pode resultar em melhorias
significativas de desempenho quando analisamos dados. Também
podemos realizar transformações nas categorias, ao mesmo tempo
que deixamos os códigos inalterados. Alguns exemplos detransformações que podem ser feitas com um custo relativamente
baixo são:
- renomear as categorias;
- concatenar uma nova categoria sem alterar a ordem ou a posição
das categorias existentes.

### Tipo Categorical do pandas

O pandas tem um tipo especial *Categorical* para armazenar dados
que utilizam a representação ou a *codificação* em categorias
baseadas em inteiros. Vamos considerar a Series usada no exemplo
anterior:

In [11]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [12]:
N = len(fruits)

In [13]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns=['basket_id', 'fruit', 'count', 'weight'])

In [14]:
df

,basket_id,fruit,count,weight
0,0,apple,7,3.723705
1,1,orange,12,1.449841
2,2,apple,13,1.435961
3,3,apple,8,3.223233
4,4,apple,10,0.373295
5,5,orange,11,1.682801
6,6,apple,12,1.507613
7,7,apple,5,3.006767


Nesse caso, *df\['fruit'\]* é um array de objetos string de Python.
Podemos convertê-lo em categorias chamando:

In [15]:
fruit_cat = df['fruit'].astype('category')

In [16]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

Os valores de *fruit_cat* não são um array NumPy, mas uma instância
de *pandas.Categorical*:

In [17]:
c = fruit_cat.values

In [18]:
type(c)

pandas.core.arrays.categorical.Categorical

O objeto *Categorical* tem os atributos *categories* e *codes*:

In [19]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [20]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

Podemos converter a coluna de um DataFrame em dados
categorizados atribuindo-lhe o resultado da conversão:

In [21]:
df['fruit'] = df['fruit'].astype('category')

In [22]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

Também é possível criar um *pandas.Categorical* diretamente a partir de
outros tipos de sequências Python:

In [23]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [24]:
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

Se você tiver obtido dados categorizados codificados de outra fonte,
poderá usar o construtor alternativo *from_codes*:

In [25]:
categories = ['foo', 'bar', 'baz']

In [26]:
codes = [0, 1, 2, 0, 0, 1]

In [27]:
my_cats_2 = pd.Categorical.from_codes(codes=codes, categories=categories)

In [28]:
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

A menos que estejam explicitamente especificadas, as conversões
de categoria não pressupõem nenhuma ordem específica para as
categorias. Assim, o array *categories* pode estar em uma ordem
diferente conforme a ordem dos dados de entrada. Ao usar
*from_codes* ou qualquer um dos demais construtores, podemos
informar se as categorias têm uma ordem significativa:

In [29]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

In [30]:
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

A saída *\[foo < bar < baz\]* indica que *'foo'* antecede *'bar'* na sequência, eassim por diante. Uma instância não ordenada de categorias pode
ser ordenada com *as_ordered*:

In [31]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

Como última observação, os dados categorizados não precisam ser
strings, apesar de eu ter mostrado somente exemplos com elas. Um
array categorizado pode ser constituído de qualquer tipo de valor
**imutável**.

### Processamentos com Categoricals

Usar *Categorical* no pandas, em comparação com a versão não
codificada (como um array de strings), em geral apresenta o mesmo
comportamento. Algumas partes do pandas, como a função *groupby*,
têm desempenho melhor quando trabalham com dados
categorizados. Há também algumas funções que podem utilizar a
flag *ordered*.

Vamos considerar alguns dados numéricos aleatórios e utilizar a
função *pandas.qcut* para uma separação em compartimentos
(binning). Essa função devolve um *pandas.Categorical*; já havíamos
usado *pandas.cut* antes no livro, mas tínhamos deixado de lado os
detalhes de como os dados categorizados funcionam:

In [32]:
np.random.seed(1234)

In [33]:
draws = np.random.randn(1000)

In [34]:
draws[:5]

array([ 0.47143516, -1.19097569,  1.43270697, -0.3126519 , -0.72058873])

Vamos compartimentar esses dados em quartis e extrair algumas
estatísticas:

In [35]:
bins = pd.qcut(draws, 4)

In [36]:
bins

[(0.0178, 0.669], (-3.565, -0.624], (0.669, 2.764], (-0.624, 0.0178], (-3.565, -0.624], ..., (0.0178, 0.669], (0.669, 2.764], (0.0178, 0.669], (0.669, 2.764], (-3.565, -0.624]]
Length: 1000
Categories (4, interval[float64]): [(-3.565, -0.624] < (-0.624, 0.0178] < (0.0178, 0.669] <
                                    (0.669, 2.764]]

Embora sejam convenientes, os quartis exatos da amostra talvez
sejam menos úteis do que seus nomes para gerar um relatório.
Podemos fazer isso com o argumento *labels* em *qcut*:

In [37]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [38]:
bins

['Q3', 'Q1', 'Q4', 'Q2', 'Q1', ..., 'Q3', 'Q4', 'Q3', 'Q4', 'Q1']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [39]:
bins.codes[:10]

array([2, 0, 3, 1, 0, 3, 3, 0, 1, 0], dtype=int8)

Os dados categorizados com rótulos em *bins* não contêm
informações sobre as fronteiras dos compartimentos em seus
dados; assim, podemos usar *groupby* para extrair algumas
estatísticas de resumo:

In [40]:
bins = pd.Series(bins, name='quartile')

In [41]:
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index())

In [42]:
results

,quartile,count,min,max
0,Q1,250,-3.563517,-0.624589
1,Q2,250,-0.624230,0.017467
2,Q3,250,0.018055,0.668488
3,Q4,250,0.669760,2.763844


A coluna *'quartile'* no resultado preserva as informações originais de
categorias, incluindo a ordem, presentes em *bins*:

In [43]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### Melhor desempenho com dados categorizados

Se você faz muitas análises em um conjunto de dados particular,
fazer a conversão para dados categorizados pode resultar, de modo
geral, em ganhos substanciais de desempenho. Além do mais, uma
versão de uma coluna de DataFrame com dados categorizados com
frequência utiliza significativamente menos memória. Vamos
considerar uma Series com 10 milhões de elementos e um número
reduzido de categorias distintas:

In [44]:
N = 10000000

In [45]:
draws = pd.Series(np.random.randn(N))

In [46]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

Vamos agora converter *labels* para dados de categoria:

In [47]:
categories = labels.astype('category')

Percebemos agora que *labels* utiliza significativamente mais memória
que categories:

In [48]:
labels.memory_usage()

80000128

In [49]:
categories.memory_usage()

10000320

A conversão para categorias obviamente não é gratuita, porém é umcusto ao qual incorremos apenas uma vez:

In [50]:
%time _ = labels.astype('category')

CPU times: user 444 ms, sys: 2.46 ms, total: 446 ms
Wall time: 445 ms


As operações de groupby podem ser significativamente mais
rápidas com dados categorizados, pois os algoritmos subjacentes
utilizam o array de códigos baseado em inteiros em vez de usar um
array de strings.

### Métodos para dados categorizados

As Series contendo dados categorizados têm vários métodos
especiais semelhantes aos métodos especializados de string
*Series.str*. Esses métodos também oferecem acesso conveniente às
categorias e aos códigos. Considere a Series a seguir:

In [51]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [52]:
cat_s = s.astype('category')

In [53]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

O atributo especial *cat* oferece acesso aos métodos de categoria:

In [54]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [55]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

Suponha que soubéssemos que o verdadeiro conjunto de categorias
para esses dados se estendesse para além dos quatro valores
observados nos dados. Podemos utilizar o método *set_categories* para
alterá-lo:

In [56]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [57]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [58]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

Embora pareça que os dados não tenham mudado, as novas
categorias se refletirão nas operações que as usam. Por exemplo,
*value_counts* respeita as categorias, se elas estiverem presentes:

In [59]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [60]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

Em conjuntos de dados grandes, dados categorizados em geral são
usados como uma ferramenta conveniente para economizar
memória e ter melhor desempenho. Depois de filtrar um DataFrame
ou uma Series grande, muitas das categorias talvez não apareçam
nos dados. Para ajudar nesse caso, podemos utilizar o método
*remove_unused_categories* e remover as categorias que não sejam
observadas:

In [61]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [62]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [63]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

Veja a Tabela Abaixo que contém uma lista dos métodos disponíveis
para categorias.

Tabela – Métodos de categoria para Series no pandas

Método | Descrição
-------|-------------------
**add_categories** | Concatena novas categorias (não usadas) no final
|das categorias existentes
**as_ordered** | Deixa as categorias ordenadas
**as_unordered** | Deixa as categorias não ordenadas
**remove_categories** | Remove categorias, definindo qualquer valor 
|removido com nulo
**remove_unused_categories** | Remove qualquer valor de categoria que não
|apareça nos dados
**rename_categories** | Substitui as categorias pelo conjunto indicado de
|novos nomes de categorias; não pode alterar o
|número de categorias
**reorder_categories** | Comporta-se como rename_categories, mas
|também pode alterar o resultado para ter categorias
|ordenadas
**set_categories** | Substitui as categorias pelo conjunto indicado de
|novas categorias; pode adicionar ou remover
|categorias

### Criando variáveis dummy para modelagem

Quando usamos ferramentas estatísticas ou de aprendizado de
máquina, com frequência transformaremos dados categorizados em
*variáveis dummy*, também conhecidas como codificação *one-hot*. Essa
operação envolve a criação de um DataFrame com uma coluna para
cada categoria distinta; essas colunas contêm 1 para as ocorrências
de uma dada categoria e 0 caso contrário.

Considere o exemplo anterior:

In [64]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

A função *pandas.get_dummies* converte esses dados categorizados unidimensionais em um DataFrame contendo a variável dummy:

In [65]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


### Uso avançado de GroupBy

###Transformações de grupos e GroupBys “não encapsulados”

No Capítulo 10, vimos o método *apply* em operações de grupo,
usado para realizar transformações. Há outro método embutido
chamado *transform*, que é semelhante a *apply*, porém impõe mais
restrições quanto ao tipo de função que você pode usar:

- pode gerar um valor escalar com o qual um broadcast será feito
para a dimensão do grupo;

- pode gerar um objeto com o mesmo formato que o grupo de
entrada;

- não deve modificar a sua entrada.

Vamos considerar um exemplo simples para demonstração:

In [66]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})

In [67]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


Eis as médias de grupo por chave:

In [70]:
g = df.groupby('key').value
g

In [71]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

Suponha que, em vez disso, quiséssemos gerar uma Series de
mesmo formato que *df\['value'\]*, mas com valores substituídos pela
média agrupada por *'key'*. Podemos passar a função *lambda x: x.mean()*
para *transform*:

In [72]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

Para as funções de agregação embutidas, podemos passar um alias
na forma de string, como no método *agg* de GroupBy:

In [73]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

Assim como *apply*, *transform* trabalha com funções que devolvem
Series, mas o resultado deve ter o mesmo tamanho que a entrada.
Por exemplo, podemos multiplicar cada grupo por 2 usando uma
função lambda:

In [74]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

Como um exemplo mais complicado, podemos calcular as
classificações em ordem decrescente para cada grupo:

In [76]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

Considere uma função de transformação de grupo composta de
agregações simples:

In [77]:
def normalize(x):
  return (x - x.mean()) / x.std()

Podemos obter resultados equivalentes nesse caso, seja usando
*transform*, seja com *apply*:

In [78]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [79]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Funções de agregação embutidas como *'mean'* ou *'sum'* em geral são
mais rápidas que uma função *apply* genérica. Elas também têm um
“passo rápido” quando usadas com *transform*, o que nos permite
executar a chamada operação de grupo *não encapsulada*:

In [80]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [81]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [82]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Embora uma operação de grupo não encapsulada possa envolver
várias agregações de grupo, a vantagem das operações vetorizadas
em geral muitas vezes compensará.

### Reamostragem de tempo em grupos

Para dados de séries temporais, o método *resample* é
semanticamente uma operação de grupo baseada em intervalos de
tempo. Eis uma pequena tabela como exemplo:

In [83]:
N = 15

In [84]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [85]:
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

In [86]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


Nesse caso, podemos indexar por *'time'* e então fazer uma
reamostragem:

In [87]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


Suponha que um DataFrame contenha várias séries temporais,
marcadas por uma coluna adicional de chave de grupo:

In [88]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})

In [89]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


Para fazer a mesma reamostragem para cada valor de *'key'*,
apresentaremos o objeto *pandas.Grouper*:

In [98]:
time_key = pd.Grouper(freq='5min')

Podemos então definir o índice de tempo, agrupar por *'key'* e *time_key*
e agregar:

In [99]:
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())

In [100]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [101]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


Uma limitação ao usar *Grouper* está no fato de o tempo ter que
ser o índice da Series ou do DataFrame.

### Técnicas para encadeamento de métodos

Ao aplicar uma sequência de transformações em um conjunto de
dados, é possível que você se veja criando diversas variáveis
temporárias que jamais serão usadas em sua análise. Considere o
exemplo a seguir:

```python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

Embora não estejamos usando nenhum dado real nesse caso, o
exemplo destaca alguns métodos novos. Inicialmente, o método
*DataFrame.assign* é uma alternativa *funcional* para atribuições de
colunas no formato *df\[k\] = v*. Em vez de modificar o objeto in-place,
ele devolve um novo DataFrame com as modificações indicadas.

Assim, as instruções a seguir são equivalentes:

```python
# Modo usual, não funcional
df2 = df.copy()
df2['k'] = v

# Modo de atribuição funcional
df2 = df.assign(k=v)
```

Fazer uma atribuição in-place pode executar mais rapidamente que
usar *assign*, porém *assign* permite encadear métodos com mais
facilidade:

```python
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
.groupby('key')
.col1_demeaned.std())
```

Utilizei os parênteses externos para que a adição de quebras de
linha fosse mais conveniente.
Um aspecto para se ter em mente ao fazer encadeamento de
métodos é que talvez você precise referenciar objetos temporários.
No exemplo anterior, não podemos referenciar o resultado de
*load_data* até que ele tenha sido atribuído à variável temporária df .
Para ajudar nesse caso, *assign* e muitas outras funções do pandas
aceitam argumentos do tipo função, também conhecidos como
*callables*.

Para mostrar as callables em ação, considere um fragmento do
exemplo anterior:

```python
df = load_data()
df2 = df[df['col2'] < 0]Esse código pode ser reescrito assim:
df = (load_data()
[lambda x: x['col2'] < 0])
```

Nesse caso, o resultado de *load_data* não é atribuído a uma variável,
de modo que a função passada em \[\] é então *vinculada* ao objeto
nessa etapa do encadeamento de métodos.
Podemos prosseguir, então, e escrever toda a sequência como uma
única expressão encadeada:

```python
result = (load_data()
[lambda x: x.col2 < 0]
.assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
.groupby('key')
.col1_demeaned.std())
```

O fato de você preferir escrever seu código nesse estilo é uma
questão de gosto pessoal, e separar a expressão em vários passos
pode deixar seu código mais legível.


### Método pipe

Podemos executar muitas tarefas com as funções embutidas do
pandas e as abordagens para encadeamento de métodos com
callables que acabamos de ver. Às vezes, porém, você terá que
usar suas próprias funções ou funções de bibliotecas de terceiros. É
em situações como essa que o método pipe entra em cena.

Considere uma sequência de chamadas de função:

```python
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

Ao usar funções que aceitem e devolvam objetos Series ou
DataFrame, podemos reescrever esse código usando chamadas a
*pipe*:

```python
result = (df.pipe(f, arg1=v1)
.pipe(g, v2, arg3=v3)
.pipe(h, arg4=v4))
```

As instruções *f(df)* e *df.pipe(f)* são equivalentes, porém *pipe* facilita a
chamada de métodos encadeados.

Um padrão possivelmente útil para *pipe* está em generalizar
sequências de operações em funções reutilizáveis. Como exemplo,
vamos considerar a subtração das médias dos grupos de uma
coluna:

```python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

Suponha que quiséssemos subtrair a média de mais de uma coluna
e alterar facilmente as chaves de grupo. Além do mais, talvez você
quisesse fazer essa transformação em uma cadeia de métodos. Eis
um exemplo de implementação:

```python
def group_demean(df, by, cols):
  result = df.copy()
  g = df.groupby(by)
  for c in cols:
  result[c] = df[c] - g[c].transform('mean')
  return result
```
Então é possível escrever o seguinte:

```python
result = (df[df.col1 < 0]
.pipe(group_demean, ['key1', 'key2'], ['col1']))
```
